In [1]:
!pip install -r requirements.txt

  Using cached Pint-0.20.1-py3-none-any.whl (269 kB)
  Using cached python_gnupg-0.5.0-py2.py3-none-any.whl (18 kB)
  Using cached eep153_tools-0.11-py2.py3-none-any.whl (4.4 kB)
  Using cached fooddatacentral-1.0.9-py3-none-any.whl
  Attempting uninstall: pint
    Found existing installation: Pint 0.17
    Uninstalling Pint-0.17:
      Successfully uninstalled Pint-0.17


In [2]:
# API key for Gov;
apikey = "bwFohFv0W79JagEjhjfy121CHf29UEljz0OYel1N"

In [3]:
# read in dietary requirements (max and min)
import pandas as pd
dri_max = pd.read_csv("Dietary Requirements Max.csv").set_index('Nutrition')

dri_min = pd.read_csv("Dietary Requirements Min.csv").set_index('Nutrition')

# convert kcal to kJ
temp = dri_max.loc['Energy']
temp.iloc[1:] = temp.iloc[1:] * 4.184
dri_max.loc['Energy'] = temp

temp = dri_min.loc['Energy']
temp.iloc[1:] = temp.iloc[1:] * 4.184
dri_min.loc['Energy'] = temp

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj._check_is_chained_assignment_possible()
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [4]:
import re
# function for age-sex specific DRI
def dri(age,sex,dietmin,dietmax):
    if age <= 3:
        index = 2

    else:
        for i, j in enumerate(list(dietmax)[3:]):
            if j[0] == sex:
                interval = re.findall(r'\d+', j)
                if len(interval) == 1:
                    if age >= int(interval[0]):
                        index = i + 3
                        break
                else:
                    if age >= int(interval[0]) and age <= int(interval[1]):
                        index = i + 3
                        break

    df = pd.DataFrame({'Nutrition': dietmin.iloc[:, 0],
                       'Max/Min': 'Minimum',
                       'Age & Sex': list(dietmin)[index],
                      'Intake': dietmin.iloc[:, index]})   
    df_max = pd.DataFrame({'Nutrition': dietmax.iloc[:, 0],
                       'Max/Min': 'Maximum',
                       'Age & Sex': list(dietmax)[index],
                      'Intake': dietmax.iloc[:, index]})   

    df = pd.concat([df, df_max], axis=0)
    
    return df
    

In [5]:
# test
dri(10, 'F', dri_min, dri_max)

,Nutrition,Max/Min,Age & Sex,Intake
Nutrition,,,,
Energy,---,Minimum,F 9-13,6694.4
Protein,RDA,Minimum,F 9-13,34.0
"Fiber, total dietary",---,Minimum,F 9-13,22.4
"Folate, DFE",RDA,Minimum,F 9-13,300.0
"Calcium, Ca",RDA,Minimum,F 9-13,1300.0
"Carbohydrate, by difference",RDA,Minimum,F 9-13,130.0
"Iron, Fe",RDA,Minimum,F 9-13,8.0
"Magnesium, Mg",RDA,Minimum,F 9-13,240.0
Niacin,RDA,Minimum,F 9-13,12.0


In [6]:
# get food list from five restaurants
food_list_total = pd.read_csv("FoodList.csv")
food_list_total = food_list_total.astype({"FDC": str})
food_list_total

,Restaurant,Dish,Ingredients,Dish_Price,Quantity,Unit,FDC,Calorie/100g,Ingredient_Price/100 gm
0,Chipotle,Chicken burrito,chicken,8.85,132.0000,gram,331960,237.0,1.320
1,Chipotle,Chicken burrito,white rice,8.85,120.0000,gram,790214,359.0,0.274
2,Chipotle,Chicken burrito,black beans,8.85,60.0000,gram,747444,180.0,0.440
3,Chipotle,Chicken burrito,lettuce,8.85,60.0000,gram,2346389,21.0,1.110
4,Chipotle,Chicken burrito,salsa,8.85,40.0000,gram,746777,34.0,0.590
...,...,...,...,...,...,...,...,...,...
315,Poke Bar,Wazzup Poke Bowl,cucumber,15.95,28.3500,gram,168409,15.0,0.880
316,Poke Bar,Wazzup Poke Bowl,green onion,15.95,7.0875,gram,170006,27.0,0.440
317,Poke Bar,Wazzup Poke Bowl,ponzu,15.95,35.4375,gram,2451144,33.0,1.900
318,Poke Bar,Wazzup Poke Bowl,wasabi,15.95,452.1825,gram,171831,292.0,8.780


In [7]:
# split food_list by restaurant
grouped = food_list_total.groupby(food_list_total.Restaurant)
restaurants = ['Chipotle', 'Thai Basil', 'Ttoust', 'IB', 'Poke Bar']

food_list_res = []

for r in restaurants: 
    food_list_res.append(grouped.get_group(r))


In [8]:
import fooddatacentral as fdc
import warnings
from collections import defaultdict

# get nutritional information for ingredients
ing_res = []
for i in range(5):
    L = []
    D = {}
    items = []
    count = 0
    food_list = food_list_res[i]
    for food in food_list.Ingredients.tolist():
        try:
            FDC = food_list.iloc[count,:].FDC
            count+=1
            temp = fdc.nutrients(apikey,FDC)
            key = temp.Units
            # convert units if necessary
            if 'Energy' in key.index:
                if key['Energy'] != 'kJ':
                    temp.Quantity['Energy'] = temp.Quantity['Energy']*4.184

            L.append(temp.Quantity)
            D[food] = temp.Quantity
            items.append(food)
        except AttributeError: 
            warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))     

    FoodNutrients_Ing = pd.DataFrame(L,dtype=float)
    FoodNutrients_Ing.index = items
    FoodNutrients_Ing = FoodNutrients_Ing.fillna(0)
    FoodNutrients_Ing = FoodNutrients_Ing.transpose() 

    #FoodNutrients_Ing

    FoodNutrients_Ing_d = pd.DataFrame(D,dtype=float)
    FoodNutrients_Ing_d = FoodNutrients_Ing_d.fillna(0)
    ing_res.append(FoodNutrients_Ing_d)

/tmp/ipykernel_30/329324121.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.Quantity['Energy'] = temp.Quantity['Energy']*4.184


In [10]:
ing_res[0]

,chicken,white rice,black beans,lettuce,salsa,sour cream,monterey jack cheese,fajita,steak,barbacoa,carnitas,guacamole,Chicken,flour tortilla,flour tortilla,flour toritlla
"Alcohol, ethyl",0.00,0.00,0.00,0.0000,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Amino acids,0.00,0.00,0.00,0.0000,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Ash,1.19,0.35,0.00,0.6063,2.3,0.7,0.0,0.00,1.02,0.00,0.00,0.00,1.19,0.00,0.00,0.00
"Boron, B",0.00,0.00,1250.00,0.0000,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Caffeine,0.00,0.00,0.00,0.0000,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Vitamin E, added",0.00,0.00,0.00,0.0000,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Vitamin K (phylloquinone),0.00,0.00,0.00,0.0000,4.2,0.5,2.5,9.40,0.00,1.40,1.10,20.30,0.00,7.20,7.20,7.20
Vitamins and Other Components,0.00,0.00,0.00,0.0000,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Water,65.30,11.60,0.00,94.2900,89.3,78.1,41.0,59.40,71.40,56.50,53.90,73.00,65.30,32.00,32.00,32.00


In [11]:
# construct table of nutritional information for dishes

dishes = food_list.groupby('Dish',sort=False)['Ingredients'].count()

FoodNutrients = pd.DataFrame(columns=dishes.keys().tolist(),
                  index=FoodNutrients_Ing.index, )
start = 0
end = 0
for i, column in enumerate(FoodNutrients): 

    FoodNutrients[column] = FoodNutrients_Ing.iloc[:, start:start+dishes[i]].sum(axis=1)
    start = start+dishes[i]
FoodNutrients

,Original Salmon Poke Bowl,Firecracker Poke Bowl,Sunset House Poke Bowl,The O.G. Poke Bowl,Goodie Mob Poke Bowl,Wazzup Poke Bowl
Proximates,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
Water,349.970000,349.370000,424.540000,336.83000,444.350000,448.360000
Energy,14278.841536,50999.296902,42197.319302,44274.83239,5629.147776,5466.693248
Protein,42.170000,30.610000,56.240000,44.55000,41.900000,33.200000
Total lipid (fat),81.880000,120.000000,71.250000,63.89000,13.850000,46.890000
...,...,...,...,...,...,...
"Fatty acids, total trans-polyenoic",0.000000,0.000000,0.000000,0.00000,0.000000,0.023000
"Fluoride, F",0.000000,1.300000,1.300000,0.00000,1.300000,1.300000
Phytosterols,0.000000,14.000000,14.000000,0.00000,14.000000,14.000000
"Sugars, added",0.000000,13.300000,13.300000,13.30000,0.000000,0.000000


In [12]:
# Convert food quantities to FDC units
food_list['FDC Quantity'] = food_list[['Quantity','Unit']].T.apply(lambda x : fdc.units(x['Quantity'],x['Unit']))

# Now may want to filter df by time or place--need to get a unique set of food names.
food_list['FDC Price'] = food_list['Dish_Price']/food_list.groupby('Dish',sort=False)['FDC Quantity'].sum()

food_list.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Dish_Prices = food_list.groupby('Dish',sort=False)['Dish_Price'].min()/food_list.groupby('Dish',sort=False)['FDC Quantity'].sum()

/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values
/tmp/ipykernel_30/2289315912.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_list['FDC Quantity'] = food_list[['Quantity','Unit']].T.apply(lambda x : fdc.units(x['Quantity'],x['Unit']))
/tmp/ipykernel_30/2289315912.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_list['FDC Price'] = food_list['Dish_Price']/food_li

In [13]:
Dish_Prices

Dish
Original Salmon Poke Bowl    3.2149155958679763 / hectogram
Firecracker Poke Bowl         6.082272748939414 / hectogram
Sunset House Poke Bowl       2.6790963298899806 / hectogram
The O.G. Poke Bowl           1.7337757450751807 / hectogram
Goodie Mob Poke Bowl         1.7472367368847699 / hectogram
Wazzup Poke Bowl             1.6819438842358623 / hectogram
dtype: object

In [14]:
# Convert food quantities to FDC units
ing_prices = []
for r in food_list_res:
    food_list = r
    food_list['FDC Quantity'] = food_list[['Quantity','Unit']].T.apply(lambda x : fdc.units(x['Quantity'],x['Unit']))

    # Now may want to filter df by time or place--need to get a unique set of food names.
    food_list['FDC Price'] = food_list['Ingredient_Price/100 gm']

    food_list.dropna(how='any') # Drop food with any missing data

    # To use minimum price observed
    Ingredient_Prices = food_list.groupby('Ingredients',sort=False)['FDC Price'].min()
    ing_prices.append(Ingredient_Prices)

/tmp/ipykernel_30/2842241641.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_list['FDC Quantity'] = food_list[['Quantity','Unit']].T.apply(lambda x : fdc.units(x['Quantity'],x['Unit']))
/tmp/ipykernel_30/2842241641.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_list['FDC Price'] = food_list['Ingredient_Price/100 gm']


In [15]:
ing_prices[4]

Ingredients
salmon            2.20
white rice        0.28
sweet onion       0.44
green onion       0.44
kale              0.88
shoyu             1.69
spicy mayo        1.38
ahi tuna          2.36
cucumber          0.88
sweet chili       1.05
house dressing    3.17
edamame           0.56
ponzu             1.90
tofu              0.74
wasabi            8.78
mayo              0.56
Name: FDC Price, dtype: float64

In [16]:
from  scipy.optimize import linprog as lp
import numpy as np
import warnings

def solve_subsistence_problem(FoodNutrients,Prices,dietmin,dietmax,max_weight=None,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    try: 
        p = Prices.apply(lambda x:x.magnitude)
    except AttributeError:  # Maybe not passing in prices with units?
        warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
        p = Prices

    p = p.dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = p.index.intersection(FoodNutrients.columns)
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)
    #print(Aall)
    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[Aall.index.intersection(dietmin.index)]
    #print(dietmin)
    #print(Amin)
    Amin = Amin.reindex(dietmin.index,axis=0)
    #print(dietmin.index)
    #print(Amin)
    idx = Amin.index.to_frame()
    #print(Amin)
    idx['type'] = 'min'
    #print(Amin)
    #Amin.index = pd.MultiIndex.from_frame(idx)
    #dietmin.index = Amin.index
    
    

    Amax = Aall.loc[Aall.index.intersection(dietmax.index)]
    Amax = Amax.reindex(dietmax.index,axis=0)
    idx = Amax.index.to_frame()
    idx['type'] = 'max'
    #Amax.index = pd.MultiIndex.from_frame(idx)
    #dietmax.index = Amax.index

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,
                   -Amax])

    b = pd.concat([dietmin,
                   -dietmax]) # Note sign change for max constraints

    # Make sure order of p, A, b are consistent
    A = A.reindex(p.index,axis=1)
    A = A.reindex(b.index,axis=0)

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
    #print(p)
    #print(A)
    #print(b)
    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='interior-point', options={'presolve': True,
                                                            'cholesky':False, 
                                                            'sym_pos':False,
                                                            'lstsq':True})

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        warnings.warn(result.message)
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result

In [52]:
# dish-based result
group = 'M 14-18'
tol = 1e-5
#FoodNutrients_t = FoodNutrients.iloc[0:10,]
#print(FoodNutrients_t)
result = solve_subsistence_problem(FoodNutrients,Dish_Prices,dri_min[group],dri_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())


Cost of diet for M 14-18 is $13.56 per day.


Diet (in 100s of grams or milliliters):
Series([], dtype: float64)


With the following nutritional outcomes of interest:
                                Outcome  Recommendation
Nutrition                                              
Energy                              NaN          9204.8
Protein                             NaN            52.0
Fiber, total dietary                NaN            30.8
Folate, DFE                         NaN           400.0
Calcium, Ca                         NaN          1300.0
Carbohydrate, by difference         NaN           130.0
Iron, Fe                            NaN            11.0
Magnesium, Mg                       NaN           410.0
Niacin                              NaN            16.0
Phosphorus, P                       NaN          1250.0
Potassium, K                        NaN          4700.0
Riboflavin                          NaN             1.3
Thiamin                             NaN         

/tmp/ipykernel_30/2046845037.py:91: UserWarning: The algorithm terminated successfully and determined that the problem is infeasible.
  warnings.warn(result.message)


In [60]:
# ingredient-based results
groups = dri_min.columns[1:]

cost_tbl = pd.DataFrame(columns = groups, index = restaurants)
diet_tbl = pd.DataFrame(columns = groups, index = restaurants)
nurt_tbl = pd.DataFrame(columns = groups, index = restaurants)
excess_tbl = pd.DataFrame(columns = groups, index = restaurants)

for group in groups:
    tol = 1e-6
    #FoodNutrients_t = FoodNutrients.iloc[0:10,]
    #print(FoodNutrients_t)
    #results_tbl = pd.dataframe(column = )
    for i in range(5):
        result = solve_subsistence_problem(ing_res[i],ing_prices[i],dri_min[group],dri_max[group],tol=tol)
        
        #results_tbl.loc[restaurants[i], group] = result
        
        print("Result for restaurant: " + restaurants[i])
        print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))
        
        cost_tbl.loc[restaurants[i], group] = result.fun
        
        # Put back into nice series
        diet = result.diet

        print("\nDiet (in 100s of grams or milliliters):")
        print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
        print()
        
        diet_tbl.loc[restaurants[i], group] = list(diet[diet >= tol])

        tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
        print("\nWith the following nutritional outcomes of interest:")
        print(tab)
        print()
        
        nurt_tbl.loc[restaurants[i], group] = list(np.abs(result.A).dot(diet))

        print("\nConstraining nutrients are:")
        excess = tab.diff(axis=1).iloc[:,1]
        print(excess.loc[np.abs(excess) < tol*100].index.tolist())
        
        excess_tbl.loc[restaurants[i], group] = list(excess.loc[np.abs(excess) < tol*100].index.tolist())

/tmp/ipykernel_30/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_30/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_30/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_30/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipy

Result for restaurant: Chipotle
Cost of diet for C 1-3 is $8.11 per day.


Diet (in 100s of grams or milliliters):
white rice     2.294679
black beans    0.906364
salsa          0.790272
sour cream     3.310155
guacamole      2.021411
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          9207.949190          4184.0
Protein                           54.705641            13.0
Fiber, total dietary              19.515728            14.0
Folate, DFE                      196.103168           150.0
Calcium, Ca                      699.999999           700.0
Carbohydrate, by difference      229.024871           130.0
Iron, Fe                           7.000000             7.0
Magnesium, Mg                    317.406858            80.0
Niacin                             7.381135             6.0
Phosphorus, P                   105

/tmp/ipykernel_30/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_30/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_30/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_30/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")


Result for restaurant: Chipotle
Cost of diet for F 4-8 is $10.03 per day.


Diet (in 100s of grams or milliliters):
white rice         2.576597
black beans        1.894298
salsa              1.097765
sour cream         3.967028
guacamole          2.374450
flour tortilla     0.009470
flour tortilla     0.009470
flour toritlla     0.009470
dtype: float64


With the following nutritional outcomes of interest:
                                     Outcome  Recommendation
Nutrition                                                   
Energy                          10460.000102          5020.8
Protein                            84.430971            19.0
Fiber, total dietary               26.753683            16.8
Folate, DFE                       235.451765           200.0
Calcium, Ca                      1000.000000          1000.0
Carbohydrate, by difference       262.644758           130.0
Iron, Fe                           12.806485            10.0
Magnesium, Mg                     523.443

/tmp/ipykernel_30/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_30/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_30/2046845037.py:91: UserWarning: The solution does not satisfy the constraints within the required tolerance of 3.16E-04, yet no errors were raised and there is no certificate of infeasibility or unboundedness. Check whether the slack and constraint residuals are acceptable; if not, consider enabling presolve, adjusting the tolerance option(s), and/or using a different method. Please consider submitting a bug report.
  warnings.warn(result.message)
/tmp/ipykernel_30/2046845037.py:22:

...

Result for restaurant: Thai Basil
Cost of diet for M 31-50 is $6.69 per day.


Diet (in 100s of grams or milliliters):
egg         4.947712
carrots     3.846580
rice        0.179987
broccoli    1.176471
peanuts     3.770187
tofu        1.708201
dtype: float64


With the following nutritional outcomes of interest:
                                     Outcome  Recommendation
Nutrition                                                   
Energy                          12970.399994          9204.8
Protein                           182.869415            56.0
Fiber, total dietary               48.402654            30.8
Folate, DFE                       901.074608           400.0
Calcium, Ca                      1000.000000          1000.0
Carbohydrate, by difference       130.000000           130.0
Iron, Fe                           22.193402             8.0
Magnesium, Mg                     777.823330           420.0
Niacin                             53.041099            16.0
Phosphorus, P 

/tmp/ipykernel_30/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_30/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_30/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_30/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipy

Result for restaurant: Ttoust
Cost of diet for F 51+ is $3.64 per day.


Diet (in 100s of grams or milliliters):
Spinach      6.432429
Pork         3.582090
potato       0.607218
Soy Sauce    0.218873
Sesame       0.264408
Rice Cake    1.195705
Flour        0.082822
dtype: float64


With the following nutritional outcomes of interest:
                                     Outcome  Recommendation
Nutrition                                                   
Energy                          12666.140068          6694.4
Protein                            67.038349            46.0
Fiber, total dietary               27.430309            22.4
Folate, DFE                       979.273733           400.0
Calcium, Ca                      1200.000001          1200.0
Carbohydrate, by difference       130.000000           130.0
Iron, Fe                           31.118617             8.0
Magnesium, Mg                     713.773890           320.0
Niacin                             15.334342         

Result for restaurant: IB
Cost of diet for M 51+ is $7.20 per day.


Diet (in 100s of grams or milliliters):
bun                0.416821
ketchup            0.221608
lettuce            1.015800
cheddar cheese     0.757351
sandwich bun       0.416821
avocado           11.906505
cheese             0.757351
dtype: float64


With the following nutritional outcomes of interest:
                                     Outcome  Recommendation
Nutrition                                                   
Energy                          12045.958131          8368.0
Protein                            75.227442            56.0
Fiber, total dietary               83.184902            28.0
Folate, DFE                      1192.022992           400.0
Calcium, Ca                      1396.454973          1000.0
Carbohydrate, by difference       130.000000           130.0
Iron, Fe                           10.424396             8.0
Magnesium, Mg                     420.000000           420.0
Niacin         

/tmp/ipykernel_30/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")
/tmp/ipykernel_30/2046845037.py:22: UserWarning: Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!
  warnings.warn("Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!")


In [57]:
cost_tbl

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Chipotle,8.105395,10.025329,10.025329,15.776241,15.770472,12366.413568,6110316.445833,162747.716275,361683.929216,755058.551358,5261873.119314,74978.830488,9301593.499273
Thai Basil,4.052806,5.681346,5.681348,7.366037,7.347198,7.559409,7.889342,6.257664,6.693508,6.257664,6.693506,7.163111,6.693506
Ttoust,1.858679,2.348786,2.348786,3.077922,3.072512,3.642299,3.685613,3.642317,3.750086,3.636711,3.730404,3.639547,3.72626
IB,5.046551,5.973081,5.97308,6.050696,6.050574,9.070692,7.207749,10.594395,7.080005,10.594395,7.198854,6.485626,7.198853
Poke Bar,6.377665,8.560379,8.560379,10.494406,10.194235,12.590013,15.451584,12.504119,15.451584,12.504119,15.451586,12.504119,15.451584


In [56]:
diet_tbl

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Chipotle,"[2.2946793771067457, 0.906363952839179, 0.7902...","[2.576596741678655, 1.894298362178777, 1.09776...",[],[],"[3.4376827015377587, 0.8139343630722031, 0.118...",[],[],[],[],[],[],[],[]
Thai Basil,"[1.6535947706889553, 1.3852752287717585, 0.709...","[2.1982570758309756, 4.11231494774835, 0.34127...","[2.198256851418826, 1.78151173378974e-06, 4.11...","[3.3071895265853226, 5.56034958804642, 0.07673...","[3.3071895423056197, 4.722821936295839, 0.1153...","[3.8562091513207064, 4.790331164253986, 0.0718...","[4.925740434841197, 0.20758255836004372, 4.847...","[3.849673202078504, 3.9686114560475874, 0.1365...","[4.947712189157788, 1.9233930755712387e-06, 3....","[3.849673201126317, 3.968611957532809, 0.13658...","[4.94771241775595, 3.8465795947305943, 0.17998...","[3.8496732024200333, 4.427298237925396, 0.0987...","[4.947712418235566, 3.8465794220684857, 0.1799..."
Ttoust,"[1.0703170302410648e-05, 2.118972292453104, 1....","[2.6412825019793043, 1.7910447678309969, 1.976...","[2.641282272586808, 1.7910446486757816, 1.9760...","[2.2248774043275987e-06, 4.399074981869411, 2....","[4.36010134416276, 2.6865671625791787, 1.01940...","[0.16673121401315355, 6.384715889044983, 3.328...","[0.23603011166510549, 6.442902795107056, 3.222...","[6.388975148971898, 3.582089541480304, 0.62883...","[0.0037044927849415035, 6.240729025680005, 3.5...","[6.39205573860163, 3.5820895521009444, 0.62730...","[6.336609607547601, 3.5820895507920705, 0.1760...","[6.432429040194432, 3.5820895514677322, 0.6072...","[6.356911669969396, 3.582088275222134, 0.21340..."
IB,"[0.12140266060050481, 0.8973065484928846, 2.49...","[0.7381884201213375, 0.36265548456572066, 0.60...","[0.7381884087079148, 0.3626555521447199, 0.607...","[0.1818589658506186, 0.9773167656716606, 0.555...","[0.18185436510852976, 0.975725026242236, 0.551...","[1.2233763054815683, 1.57576435130077e-06, 2.1...","[0.5000167883018362, 0.025090358481825042, 1.0...","[1.5156130817054203, 3.8785864811010535, 0.173...","[0.5156678917103827, 0.31452057376256354, 1.20...","[1.5156130812067994, 3.8785864897315094, 0.173...","[0.41682131618836893, 0.22160690879118547, 1.0...","[0.6185488197692462, 0.7183637701925146, 0.656...","[0.41682140001715734, 0.2216078528273798, 1.01..."
Poke Bar,"[0.2786377700974017, 1.04206165823365, 4.13955...","[0.3715170277047409, 0.923279850987226, 6.4896...","[0.37151702781763934, 0.9232798518582935, 6.48...","[0.6315231559302386, 0.8115626100037986, 1.611...","[0.593710832494077, 0.7905590873998726, 1.4035...","[1.8430382611738536, 0.8342380454859687, 7.086...","[0.6937842576074815, 0.48377537465033155, 12.9...","[1.8802723382680828, 0.8456054989862727, 6.898...","[0.6937842973729921, 0.4837754199649512, 12.93...","[1.8802723187799568, 0.8456055203133529, 6.898...","[0.6937844327917748, 0.4837758079467785, 2.984...","[1.880272337983991, 0.8456054990230503, 6.8981...","[0.693784262838222, 0.4837754128303332, 12.933..."


In [48]:
nurt_tbl.iloc[1,1]

[10459.999798933926,
 148.0204034319321,
 43.72933421281672,
 752.0014431693484,
 1000.0000005891244,
 130.00000122630277,
 18.003190665709916,
 633.4400702535934,
 45.490381594739276,
 1832.0487680555366,
 3800.0014301244255,
 1.7853951206563616,
 2.563432257431894,
 400.0000014840746,
 2.242222232640391,
 1.94665134375487,
 52.64301173914858,
 20.721591347314654,
 55.000000539476886,
 18.269350333274893,
 794.8684594554832,
 10459.999798933926]

In [49]:
excess_tbl

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Chipotle,"[Calcium, Ca, Iron, Fe, Riboflavin, Thiamin, V...","[Calcium, Ca, Niacin, Riboflavin, Thiamin, Vit...",[],[],"[Calcium, Ca, Iron, Fe, Niacin, Riboflavin, Th...",[],[],[],[],[],[],[],[]
Thai Basil,"[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Calcium, Ca, Carbohydrate, by difference, Pot..."
Ttoust,"[Carbohydrate, by difference, Potassium, K, Ri...","[Carbohydrate, by difference, Potassium, K, Ri...","[Carbohydrate, by difference, Potassium, K, Ri...","[Carbohydrate, by difference, Potassium, K, Ri...","[Carbohydrate, by difference, Potassium, K, Ri...","[Carbohydrate, by difference, Phosphorus, P, P...","[Carbohydrate, by difference, Niacin, Phosphor...","[Fiber, total dietary, Calcium, Ca, Potassium,...","[Fiber, total dietary, Potassium, K, Riboflavi...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Fiber, total dietary, Niacin, Potassium, K, R...","[Calcium, Ca, Carbohydrate, by difference, Pot...","[Carbohydrate, by difference, Niacin, Potassiu..."
IB,"[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Iron, Fe, Thiamin, Sodium, Na, E...","[Calcium, Ca, Carbohydrate, by difference, Iro...","[Calcium, Ca, Iron, Fe, Thiamin, Sodium, Na, E...","[Carbohydrate, by difference, Magnesium, Mg, T...","[Calcium, Ca, Iron, Fe, Thiamin, Sodium, Na, E...","[Carbohydrate, by difference, Magnesium, Mg, T...","[Calcium, Ca, Carbohydrate, by difference, Thi...","[Carbohydrate, by difference, Magnesium, Mg, T..."
Poke Bar,"[Carbohydrate, by difference, Potassium, K, Th...","[Carbohydrate, by difference, Potassium, K, Th...","[Carbohydrate, by difference, Potassium, K, Th...","[Carbohydrate, by difference, Phosphorus, P, P...","[Carbohydrate, by difference, Potassium, K, Th...","[Carbohydrate, by difference, Thiamin, Vitamin...","[Carbohydrate, by difference, Vitamin B-12, Vi...","[Carbohydrate, by difference, Potassium, K, Th...","[Carbohydrate, by difference, Vitamin B-12, Vi...","[Carbohydrate, by difference, Potassium, K, Th...","[Carbohydrate, by difference, Vitamin B-12, Vi...","[Carbohydrate, by difference, Potassium, K, Th...","[Carbohydrate, by difference, Vitamin B-12, Vi..."


In [64]:
# examine effects of price change on subsistence diet cost for T-toust, the cheapest option
# for men and women aged 19-30
import cufflinks as cf
cf.go_offline()

group_pc = ['F 19-30', 'M 19-30']

for group in group_pc: 
    scale = [.5,.6,.7,.8,.9,1.,1.1,1.2,1.3,1.4,1.5]

    cost0 = solve_subsistence_problem(ing_res[2],ing_prices[2],dri_min[group],dri_max[group],tol=tol).fun

    Price_response={}
    for s in scale:
        cost = {}
        for i,p in enumerate(ing_prices[2]):
            my_p = ing_prices[2].copy()
            my_p[i] = p*s
            result = solve_subsistence_problem(ing_res[2],my_p,dri_min[group],dri_max[group],tol=tol)
            cost[ing_prices[2].index[i]] = np.log(result.fun/cost0)
        Price_response[np.log(s)] = cost

    Price_response = pd.DataFrame(Price_response).T
    Price_response.iplot(xTitle='change in log price',yTitle='change in log cost')

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!



In [82]:
# examine effects of price change on subsistence diet cost for T-toust, the cheapest option
# for men and women aged 19-30
import cufflinks as cf
cf.go_offline()

ReferenceGood = 'Squash'

group_pc = ['F 19-30', 'M 19-30']

for group in group_pc: 

    scale = [0.5,0.75,0.9,1.,1.1,1.2,1.3,1.4,1.5,2,4]

    cost0 = solve_subsistence_problem(ing_res[2],ing_prices[2],dri_min[group],dri_max[group],tol=tol).fun

    my_p = ing_prices[2].copy()
    print(my_p)
    diet = {}
    for s in scale:

        my_p[ReferenceGood] = ing_prices[2][ReferenceGood]*s
        result = solve_subsistence_problem(ing_res[2],ing_prices[2],dri_min[group],dri_max[group],tol=tol)
        diet[my_p[ReferenceGood]] = result.diet

    Diet_response = pd.DataFrame(diet).T
    Diet_response.index.name = '%s Price' % ReferenceGood

    Diet_response.reset_index(inplace=True)

    # Get rid of units for index (cufflinks chokes)
    Diet_response['%s Price' % ReferenceGood] = Diet_response['%s Price' % ReferenceGood].apply(lambda x: x.magnitude)

    Diet_response = Diet_response.set_index('%s Price' % ReferenceGood)

    # Just look at goods consumed in quantities greater than error tolerance
    Diet_response.loc[:,(Diet_response>tol).sum()>0].iplot(xTitle='%s Price' % ReferenceGood,yTitle='Hectograms')

/tmp/ipykernel_30/2046845037.py:22: UserWarning:

Prices have no units.  BE CAREFUL!  We're assuming prices are per hectogram or deciliter!



Ingredients
Beef                 1.5430
Egg                  0.5000
Carrots              0.1800
Purple Cabbage       0.3700
Squash               0.4600
Spinach              0.3100
Jasmine Rice         0.2800
Kimchi               0.0600
Onions               0.2926
Fish Sauce           0.0300
Pork                 0.3980
Tofu                 0.7400
Green Onions         0.3400
Chili                1.7200
Beef Short Ribs      2.4200
potato               0.1800
fish cake            0.3200
kimchi               0.0600
Soy Sauce            0.0200
Sesame               0.2000
Brown Sugar          0.2200
Lettuce              0.2800
Rice Cake            0.0300
Sweet Chili Sauce    0.2926
Glass Noodle         0.1200
Seaweed              0.7200
Cabbage              0.2200
Flour                0.2100
Chicken              0.4400
Teriyaki Sauce       0.4500
Name: FDC Price, dtype: float64


AttributeError: 'float' object has no attribute 'magnitude'

0.28